In [2]:
from dotenv import load_dotenv
load_dotenv()

%load_ext google.cloud.bigquery

In [3]:
%%bigquery top_1000_github_users
SELECT u.login, u.company, u.state, u.city, count(f.user_id) as follower_count
FROM `ghtorrent-bq.ght_2018_04_01.followers` f
JOIN `ghtorrent-bq.ght_2018_04_01.users` u ON f.follower_id = u.id
GROUP BY f.follower_id, u.login, u.company, u.fake, u.state, u.city
order by follower_count desc
LIMIT 1000

,follower_id,login,company,fake,state,city,follower_count
0,5203,torvalds,Linux Foundation,False,OR,Portland,80184
1,896,JakeWharton,"Google, Inc.",False,PA,Pittsburgh,48120
2,616741,ruanyf,None,False,None,Shanghai,39102
3,376498,Tj,Apex,False,BC,Victoria,37402
4,6240,addyosmani,Google,False,CA,Mountain View,32666
5,1779,paulirish,"Google Chrome, ♥z",False,CA,Palo Alto,29690
6,18556,yyx990803,None,False,None,None,29200
7,417948,gaearon,@facebook,False,England,London,27415
8,3871,sindresorhus,@avajs @chalk @yeoman,False,None,None,25701
9,9236,mojombo,None,False,CA,San Francisco,25112


In [43]:
import requests
import json
import os

GITHUB_URL = "https://api.github.com/"

"""
Function to get JSON response from a URL
:params:
    url     string
:return:
            JSON
"""
def __get_json_response(url):
    headers = {'Authorization': 'token ' + os.environ['GITHUB_API_KEY']}
    response = requests.get(url, headers=headers)
    return json.loads(response.text)

"""
Function to add email to a set of emails and set a loop break flag
:params:
    email_set       set         set of all the emails for the user
    email           string      new email to be added
    max_len         integer     maximum number of emails to be extracted
:return:
    email_set       set         set of all the emails for the user
    break_flag      boolean     if max_limit is reached, break_flag is set to True
"""
def __add_email(email_set, email, max_len):
    email_set = email_set | set([email])
    break_flag = (len(email_set) >= max_len)

    return email_set, break_flag

"""
Function to get user emails using GitHub APIs
:params:
    user        string      a valid GitHub username
    max_limit   integer     maximum number of email ID to be fetched
:return:
    user_email  set         a set of all emails extracted
    message     string      if any error occurs, this holds the respective error message
"""
def __get_github_emails(user, max_limit):
    user_email = set([])
    break_flag = False
    try:
        users_profile_url = GITHUB_URL + "users/{0}".format(user)
        response = __get_json_response(users_profile_url)

        # some error encountered
        if 'message' in response:
            if response['message'] == 'Not Found':
                return u'You need to enter a valid GitHub Username'
            else:
                return response['message']
        print(response)
        name = response['name']
        user_name = response['name']
        company = response['company']
        # if user has a public email, add that to the set of emails
        if response['email']:
            user_email, break_flag = __add_email(user_email, response['email'], max_limit)

        if not break_flag:
            users_repository_url = GITHUB_URL + "users/{0}/repos?type=owner&sort=updated".format(user)
            response = __get_json_response(users_repository_url)

            for repo in response:
                if not repo['fork']:
                    users_repository_name = repo['full_name']
                    repos_commit_url = GITHUB_URL + "repos/{0}/commits".format(users_repository_name)
                    commit_reponse = __get_json_response(repos_commit_url)
                    
                    possible_positions = ['committer', 'author']

                    for commit in commit_reponse:
                        for i in possible_positions:
                            if commit['commit'][i]['name'] == user_name:
                                email_string = commit['commit'][i]['email']
                                if "noreply" not in email_string:
                                    user_email, break_flag = __add_email(user_email, email_string, max_limit)

                        if break_flag:
                            break

                if break_flag:
                    break

        if len(user_email) > 0:
            return user_email, name, company
        else:
            return u'No emails found', u'No first name found', u'No company found'

    except requests.exceptions.ConnectionError:
        return u'Proper internet connection not found', u'Proper internet connection not found', u'Proper internet connection not found'

"""
Function to get the emails associated to a username on GitHub
:params:
    username    string      a valid GitHub username
    num         integer     maximum number of email ID to be fetched, default 1
:return:
    response    JSON response
        success     boolean     flag to determine other key in JSON
        email       list        if 'success' is True, list of all the emails fetched
        message     string      if 'success' is False, returns the error message
"""
def get(username, num=1):
    email, name, co = __get_github_emails(username, num)
    if type(email) == set:
        response = {
            'success' : True,
            'data': [email.pop(), name, co]
        }
    else:
        response = {
            'success' : False,
            'message' : [email, name, co]
        }

    return response

In [44]:
import pandas as pd
result = pd.DataFrame()
for index, row in top_1000_github_users.iterrows():
    print(index)
    login            = row['login']
    company          = row['company']
    state            = row['state']
    city             = row['city']
    follower_count   = row['follower_count'] 
    try:
        resp = get(login)
        if 'data' in resp:
            email, name, co  =resp['data']
            row['email'] = email
            row['name'] = name
            row['company'] = co
            result = result.append(row, ignore_index=True)
    except Exception as e:
        print(e)
    


0
{u'public_repos': 6, u'site_admin': False, u'subscriptions_url': u'https://api.github.com/users/torvalds/subscriptions', u'gravatar_id': u'', u'hireable': None, u'id': 1024025, u'followers_url': u'https://api.github.com/users/torvalds/followers', u'following_url': u'https://api.github.com/users/torvalds/following{/other_user}', u'blog': u'', u'followers': 87340, u'location': u'Portland, OR', u'type': u'User', u'email': None, u'bio': None, u'gists_url': u'https://api.github.com/users/torvalds/gists{/gist_id}', u'company': u'Linux Foundation', u'events_url': u'https://api.github.com/users/torvalds/events{/privacy}', u'html_url': u'https://github.com/torvalds', u'updated_at': u'2019-01-23T02:49:04Z', u'node_id': u'MDQ6VXNlcjEwMjQwMjU=', u'received_events_url': u'https://api.github.com/users/torvalds/received_events', u'starred_url': u'https://api.github.com/users/torvalds/starred{/owner}{/repo}', u'public_gists': 0, u'name': u'Linus Torvalds', u'organizations_url': u'https://api.github.

KeyboardInterrupt: 

In [41]:

result.to_csv('top_1000_github_users.csv', encoding = 'utf-8')

In [42]:
!open top_1000_github_users.csv

In [39]:
result[result.columns]

,city,company,email,fake,follower_count,follower_id,login,name,state
0,Portland,Linux Foundation,torvalds@linux-foundation.org,0.0,80184.0,5203.0,torvalds,Linus Torvalds,OR
1,Pittsburgh,"Google, Inc.",jakewharton@gmail.com,0.0,48120.0,896.0,JakeWharton,Jake Wharton,PA
2,Shanghai,None,yifeng.ruan@gmail.com,0.0,39102.0,616741.0,ruanyf,Ruan YiFeng,None
3,Victoria,Apex,tj@apex.sh,0.0,37402.0,376498.0,Tj,TJ Holowaychuk,BC
4,Mountain View,Google,addyosmani@gmail.com,0.0,32666.0,6240.0,addyosmani,Addy Osmani,CA
5,Palo Alto,"Google Chrome, ♥z",paul.irish@gmail.com,0.0,29690.0,1779.0,paulirish,Paul Irish,CA
6,None,None,yyx990803@gmail.com,0.0,29200.0,18556.0,yyx990803,Evan You,None
7,London,@facebook,dan.abramov@me.com,0.0,27415.0,417948.0,gaearon,Dan Abramov,England
8,None,@avajs @xojs @chalk,sindresorhus@gmail.com,0.0,25701.0,3871.0,sindresorhus,Sindre Sorhus,None
9,San Francisco,None,tom@mojombo.com,0.0,25112.0,9236.0,mojombo,Tom Preston-Werner,CA
